In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rnd
from xgboost import XGBRegressor
from scipy.stats.stats import pearsonr   
from sklearn.metrics import mean_absolute_error

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from scipy.stats import skew  # for some statistics
from lightgbm import LGBMRegressor
from scipy.special import boxcox1p
from sklearn.pipeline import make_pipeline
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

X = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
X_test_full = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')


In [3]:
from scipy.stats import norm
from scipy.stats import multivariate_normal as mn

def z_score(df, threshold=1):
    mean, std = np.mean(df), np.std(df)
    z_score = np.abs((df - mean) / std)
    good = z_score < threshold
    print(f"Le Z-score de {threshold} correspond à une probabilité de {100 * 2 * norm.sf(3):0.2f}%")
    print(f"Rejection {(~good).sum()} points")
    return good

In [4]:
def featureEngineering(data, test_data):
        
    # Removing outliers
    filteredData = z_score(data['GrLivArea'], 3)
    data = data[filteredData]
    
    data.drop(['PoolQC'], axis=1, inplace=True)
    data.drop(['Utilities', 'Street'], axis=1, inplace=True)
    
    y = data.SalePrice
    data.drop(['SalePrice'], axis=1, inplace=True)
    
    XConc = pd.concat([data, test_data]).reset_index(drop=True)

    objects = []
    for i in XConc.columns:
        if XConc[i].dtype == object:
            objects.append(i)
    XConc.update(XConc[objects].fillna('None'))

    XConc['LotFrontage'] = XConc.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    XConc['GarageYrBlt'] = XConc.groupby('YearBuilt')['GarageYrBlt'].transform(lambda x: x.fillna(x.median())) # -216 sur la MAE

    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerics = []
    for i in XConc.columns:
        if XConc[i].dtype in numeric_dtypes:
            numerics.append(i)
    data.update(XConc[numerics].fillna(0))
    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerics2 = []
    
    for i in XConc.columns:
        if XConc[i].dtype in numeric_dtypes:
            numerics2.append(i)
    skew_features = XConc[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

    high_skew = skew_features[skew_features > 0.5]
    skew_index = high_skew.index

    for i in skew_index:
        XConc[i] = boxcox1p(XConc[i], boxcox_normmax(XConc[i] + 1))
        
    for col in numerics:
        if np.any(np.isnan(XConc[col])):
            XConc[col] = XConc[col].transform(lambda x: x.fillna(x.median()))

    XConc['YrBltAndRemod']=XConc['YearBuilt']+XConc['YearRemodAdd']
    XConc['FullSF']=XConc['TotalBsmtSF'] + XConc['1stFlrSF'] + XConc['2ndFlrSF']


    XConc['BsmtAndHouseSF'] = (XConc['BsmtFinSF1'] + XConc['BsmtFinSF2'] +  XConc['1stFlrSF'] + XConc['2ndFlrSF'])

    XConc['BathroomsCount'] = (XConc['FullBath'] + (0.5 * XConc['HalfBath']) + XConc['BsmtFullBath'] + (0.5 * XConc['BsmtHalfBath']))

    XConc['PorchSF'] = (XConc['OpenPorchSF'] + XConc['3SsnPorch'] + XConc['EnclosedPorch'] + XConc['ScreenPorch'] +
                          XConc['WoodDeckSF'])

    XConc['haspool'] = XConc['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    XConc['has2ndfloor'] = XConc['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
    XConc['hasgarage'] = XConc['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    XConc['hasbsmt'] = XConc['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
    XConc['hasfireplace'] = XConc['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

    # TODO : trouver des groupby?

    
    return XConc, y

In [5]:
# XGBoost
my_model_2 = XGBRegressor(eta='0.001', n_estimators=1000, learning_rate=0.03, random_state=0, min_child_weight=7, max_depth=5, n_jobs=-1, subsample=0.55,
                              colsample_bytree=0.2, reg_lambda=3.5, alpha=110000)

resXConc, y = featureEngineering(X, X_test_full)
final_features = pd.get_dummies(resXConc).reset_index(drop=True)

#print(np.any(np.isnan(final_features)))

X = final_features.iloc[:len(y), :]
X_sub = final_features.iloc[len(X):, :]

#my_model_2.fit(X, y)


Le Z-score de 3 correspond à une probabilité de 0.27%
Rejection 16 points


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/opt/conda/lib/python3.7/site-

In [6]:



pred1 = np.zeros(X.shape[0])
pred2 = np.zeros(X.shape[0])
pred3 = np.zeros(X.shape[0])
pred4 = np.zeros(X.shape[0])

test1 = np.zeros(X_sub.shape[0])
test2 = np.zeros(X_sub.shape[0])
test3 = np.zeros(X_sub.shape[0])
test4 = np.zeros(X_sub.shape[0])

params_lgb = {'reg_alpha': 4.973064761998367, 'reg_lambda': 0.06365096912006087,'colsample_bytree': 0.24,
              'subsample': 0.8, 'learning_rate': 0.015, 'max_depth': 100, 'num_leaves': 43,'min_child_samples': 141,
              'cat_smooth': 18,'metric': 'rmse', 'random_state': 48,'n_estimators': 40000}

kf = KFold(n_splits=5,random_state=48,shuffle=True)
n=0

for trn_idx, test_idx in kf.split(X, y):
    print(f"fold: {n+1}")
    X_tr,X_val = X.iloc[trn_idx],X.iloc[test_idx]
    y_tr,y_val= y.iloc[trn_idx] , y.iloc[test_idx]
    
    
    model1 = lgb.LGBMRegressor()
    model1.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=200,verbose=False)
    pred1[test_idx] = model1.predict(X_val)
    test1 += model1.predict(X_sub)/kf.n_splits
    rmse1 = mean_squared_error(y_val, model1.predict(X_val), squared=False)
    print(": model1 rmse = {}".format(rmse1))
    
    model2 = RandomForestRegressor(random_state=1)
    model2.fit(X_tr,y_tr)
    pred2[test_idx] = model2.predict(X_val)
    test2 += model2.predict(X_sub)/kf.n_splits
    rmse2 = mean_squared_error(y_val, model2.predict(X_val), squared=False)
    print(": model2 rmse = {}".format(rmse2))    

    model3 = Ridge()
    model3.fit(X_tr,y_tr)
    pred3[test_idx] = model3.predict(X_val)
    test3 += model3.predict(X_sub)/kf.n_splits
    rmse3 = mean_squared_error(y_val, model3.predict(X_val), squared=False)
    print(": model3 rmse = {}".format(rmse3))
    
    model4 = XGBRegressor(n_estimators=800, learning_rate=0.03, random_state=0, min_child_weight=7, max_depth=5, n_jobs=-1, subsample=0.5)
    model4.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=200,verbose=False)
    pred4[test_idx] = model4.predict(X_val)
    test4 += model4.predict(X_sub)/kf.n_splits
    rmse4 = mean_squared_error(y_val, model4.predict(X_val), squared=False)
    print(": model4 rmse = {}".format(rmse4))
    print(": average all models rmse = {}".format((rmse1+rmse2+rmse3+rmse4)/4))

    n+=1
    
stacked_predictions = np.column_stack((pred1,pred2,pred3, pred4))
stacked_test_predictions = np.column_stack((test1,test2,test3, test4))

l1_train = pd.DataFrame(data={
    "lgbm": pred1.tolist(),
    "RandomForestRegressor": pred2.tolist(),
    "LinearRegression": pred3.tolist(),
    "xgb": pred4.tolist(),
    "target": y
    })
l1_test = pd.DataFrame(data={
    "lgbm": test1.tolist(),
    "RandomForestRegressor": test2.tolist(),
    "LinearRegression": test3.tolist(),
    "xgb": test4.tolist()
    })

kf = KFold(n_splits=5,random_state=48,shuffle=True)
final_prediction = np.zeros(X_sub.shape[0])
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(stacked_predictions, y):
    X_tr,X_val=stacked_predictions[trn_idx],stacked_predictions[test_idx]
    y_tr,y_val=y.iloc[trn_idx],y.iloc[test_idx]
    
    meta_model = Lasso(alpha =0.00001)
    
    meta_model.fit(X_tr,y_tr)
    
    final_prediction +=meta_model.predict(stacked_test_predictions)/kf.n_splits
    rmse.append(mean_squared_error(y_val, meta_model.predict(X_val), squared=False))
    print(f"fold: {n+1}, rmse: {rmse[n]}")
    n+=1

fold: 1


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


: model1 rmse = 20547.424285851932
: model2 rmse = 23047.835911295777
: model3 rmse = 22325.8505377069


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


: model4 rmse = 18359.164613009365
: average all models rmse = 21070.068836966
fold: 2
: model1 rmse = 29213.130567364915


...

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


: model1 rmse = 21124.183020751472
: model2 rmse = 23249.476299410468
: model3 rmse = 19015.91433860506


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


: model4 rmse = 20218.601476478718
: average all models rmse = 20902.04378381143
fold: 1, rmse: 18618.048545591657
fold: 2, rmse: 25748.420855484525
fold: 3, rmse: 23732.715708581683
fold: 4, rmse: 16736.186361600772
fold: 5, rmse: 18290.436236051657


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.584e+11, tolerance: 6.571e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.132e+11, tolerance: 5.831e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

In [7]:
#scores = -1 * cross_val_score(my_model_2, X, y,
#                              cv=5,
#                              scoring='neg_mean_absolute_error')
#print("Average MAE score (across experiments):")
#print(scores.mean())

In [8]:
params = {'colsample_bytree': [0.2],
          'lambda': [3.5],
          'learning_rate': [0.03],
          'n_estimators' : [1000]
           }

xgb1 = XGBRegressor(n_estimators=800, learning_rate=0.03, random_state=0, min_child_weight=7, max_depth=5, n_jobs=-1, subsample=0.5)



gcv = GridSearchCV(xgb1, param_grid=params, cv=2, n_jobs=-1)
gcv.fit(X, y)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",gcv.best_estimator_)
print("\n The best score across ALL searched params:\n",gcv.best_score_)
print("\n The best parameters across ALL searched params:\n",gcv.best_params_)


 Results from Grid Search 

 The best estimator across ALL searched params:
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.2,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='', lambda=3.5,
             learning_rate=0.03, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=7,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, ...)

 The best score across ALL searched params:
 0.9100205001205651

 The best parameters across ALL searched params:
 {'colsample_bytree': 0.2, 'lambda': 3.5, 'learning_rate': 0.03, 'n_estimators': 1000}


In [9]:


# Submit
#predsSub = my_model_2.predict(X_sub)
output = pd.DataFrame({'Id': X_sub.Id,
                       'SalePrice': final_prediction})
output.to_csv('submission.csv', index=False)

